In [1]:
import numpy as np
import MDAnalysis as mda
import os
from numpy.linalg import norm
from scipy.optimize import curve_fit
from MDAnalysis.analysis.rms import rmsd
#import matplotlib.pyplot as plt
from scipy import stats
import math
#import seaborn
import pandas as pd
#import bokeh

In [2]:
os.chdir('/run/media/smuraru/Tank1/GO/ss_o1/')

In [3]:
u = mda.Universe('md_2.tpr', 'md_2.trr')

In [4]:
#Selections
sel_DNA_not_backbone = u.select_atoms('nucleic and not nucleicbackbone and not resname GGG')
sel_DNA_backbone = u.select_atoms('nucleicbackbone and not resname GGG')
sel_MG = u.select_atoms('name MG')
sel_CL = u.select_atoms('name CL')

In [ ]:
sel_DNA_backbone

In [ ]:
sel_indep = {}
for residue in sel_DNA_not_backbone.residues:
    for resid in sel_DNA_not_backbone.resids:
        if ( (str(residue)[9:-7] + ' ' + str(resid)) not in sel_indep):
            sel_indep[str(residue)[9:-7] + ' ' + str(resid)] = sel_DNA_not_backbone.select_atoms('resname ' + str(residue)[9:-7] + ' and resid ' + str(resid))
print(len(sel_indep))

sel_indep_copy = sel_indep.copy()
for key in sel_indep_copy.keys():
        if (len(sel_indep_copy[key]) == 0):
            sel_indep.pop(key)
print(len(sel_indep))

In [5]:
backbone_indep = {}
for residue in sel_DNA_backbone.residues:
    for resid in sel_DNA_not_backbone.resids:
        if ((str(residue)[9:-7] + ' ' + str(resid)) not in backbone_indep):
            backbone_indep[str(residue)[9:-7] + ' ' + str(resid)] = sel_DNA_backbone.select_atoms('resname ' + str(residue)[9:-7] + ' and resid ' + str(resid))
print(len(backbone_indep))

backbone_indep_copy = backbone_indep.copy()
for key in backbone_indep_copy.keys():
    if (len(backbone_indep_copy[key]) == 0):
        backbone_indep.pop(key)
print(len(backbone_indep))

72
12


In [6]:
sel_CX = u.select_atoms('resname GGG') # or resname E1A or resname H1A or resname C1A or resname P1A')

In [ ]:
sel_CX

In [ ]:
sel_indep.keys()

In [ ]:
backbone_indep.keys()

In [7]:
CX_x = [sel_CX.positions[i][0] for i in range(len(sel_CX.positions))]
CX_y = [sel_CX.positions[i][2] for i in range(len(sel_CX.positions))]
CX_x2D = np.asarray(CX_x)
CX_y2D = np.asarray(CX_y)
slope_CX, intercept_CX, r_value_CX, p_value_CX, std_err_CX = stats.linregress(CX_x2D, CX_y2D)

In [8]:
intercept_CX

25.931123395519926

In [ ]:
angle = {}
x = {}
y = {}
dist = {}
for key in sel_indep.keys():
    angle[str(key)] = {}
    x[key] = []
    y[key] = []
    dist[str(key)] = {}
    print("key: ", key)
    for frame in u.trajectory:
        x[key] = [sel_indep[key].positions[i][0] for i in range(len(sel_indep[key].positions))]
        y[key] = [sel_indep[key].positions[i][2] for i in range(len(sel_indep[key].positions))]
        x2D = np.asarray(x[key])
        y2D = np.asarray(y[key])
        slope_DNA_base, intercept_DNA_base, r_value_DNA_base, p_value_DNA_base, std_err_DNA_base = stats.linregress(x2D, y2D)
        common_x = float(intercept_DNA_base - intercept_CX) / (slope_CX - slope_DNA_base)
        dist_DNA = math.sqrt((slope_DNA_base * common_x)**2 + common_x**2)
        dist_CX = math.sqrt((slope_CX * common_x) + common_x**2)
        dist_3 = abs(intercept_DNA_base - intercept_CX)
        theta = np.arccos((dist_DNA**2 + dist_CX**2 - dist_3**2) / (2 * dist_DNA * dist_CX))
        angle[str(key)][int(frame.frame)] = np.rad2deg(theta)
        mobile = np.array([sel_indep[key].center_of_geometry()[2]])
        ref = np.array([sel_CX.center_of_geometry()[2]])
        dist[str(key)][int(frame.frame)] = rmsd(mobile, ref)

In [ ]:
frame_x = np.asarray([int(str(frame.frame)) for frame in u.trajectory])
for key in sel_indep.keys():
    dist_array = np.asarray([dist[str(key)][int(frame.frame)] for frame in u.trajectory])
    angle_array = np.asarray([angle[str(key)][int(frame.frame)] for frame in u.trajectory])
    
    fig1 = plt.figure()
    fig1.suptitle('Angle formed by residue ' + str(key) + ' with the graphene molecule', fontsize=14, fontweight='bold')
    plt.bar(frame_x, angle_array)
   # fig1.set_xlabel('Frame')
  #  fig1.set_ylabel('Degrees')
    #plt.xticks(frame_x, str(frame_x))
    plt.show()
    
    fig2 = plt.figure()
    fig2.suptitle('Distance between residue ' + str(key) + ' and the graphene molecule', fontsize=14, fontweight = 'bold')
    plt.plot(frame_x, dist_array)
   # fig2.set_xlabel('Frame')
    #fig2.set_ylabel('Distance (Å)')
   # plt.xticks(frame_x, str(frame_x))
    plt.show()
    


In [ ]:
for key in sel_indep.keys():
    if (dist[str(key)][0]<=15 ):
        print("Key: ", key, " Dist: ", dist[str(key)][0], " Angle: ", angle[str(key)][0])
    else:
        print("Not within adsorbtion: ", key, " Dist: ", dist[str(key)][0], " Angle: ", angle[str(key)][0])

In [ ]:
for key in sel_indep.keys():
    if (dist[str(key)][10]<=6 ):
        print("Key: ", key, " Angle: ", angle[str(key)][10])
    else:
        print("Not adsorbed: ", key, " Dist: ", dist[str(key)][10], " Angle: ", angle[str(key)][10])

In [ ]:
backbone_adsorbed = backbone_indep.copy()
for key in backbone_indep.keys():
    if (str(key) != 'DC3 7901') and (str(key) != 'DG 7892') and (str(key) != 'DA 7895') and (str(key) != 'DA 7891') and (str(key) != 'DA 7890') and (str(key) != 'DA 7888') and (str(key) != 'DC 7897') and (str(key) != 'DT 7898') and (str(key) != 'DT 7896') and (str(key) != 'DT 7894') and (str(key) != 'DT 7889'):
        backbone_adsorbed.pop(key)
#backbone_adsorbed['DC 7899'] = backbone_indep['DC 7899']
#backbone_adsorbed['DC 7900'] = backbone_indep['DC 7900']
#backbone_adsorbed['DC3 7901'] = backbone_indep['DC3 7901']
backbone_adsorbed.keys()

In [ ]:
x_ad = {}
y_ad = {}
angle_ad = {}
dist_ad = {}
for frame in u.trajectory:
    x_ad[int(frame.frame)] = []
    y_ad[int(frame.frame)] = []
    for key in backbone_adsorbed.keys():
        for i in range(len(backbone_indep[key].positions)):
            x_ad[int(frame.frame)].append(backbone_indep[key].positions[i][0])
            y_ad[int(frame.frame)].append(backbone_indep[key].positions[i][2])
  #      x_ad.append(float(backbone_indep[key].positions[i][0]) for i in range(len(backbone_indep[key].positions)))
  #      y_ad.append(float(backbone_indep[key].positions[i][2]) for i in range(len(backbone_indep[key].positions)))
    x2D_ad = np.asarray(x_ad[int(frame.frame)])
    y2D_ad = np.asarray(y_ad[int(frame.frame)])
    slope_backbone_ad, intercept_backbone_ad, r_value_backbone_ad, p_value_backbone_ad, std_err_backbone_ad = stats.linregress(x2D_ad, y2D_ad)
    common_x_ad = float(intercept_backbone_ad - intercept_CX) / (slope_CX - slope_backbone_ad)
    dist_backbone_ad = math.sqrt((slope_backbone_ad * common_x_ad)**2 + common_x_ad**2)
    dist_CX = math.sqrt((slope_CX * common_x_ad) + common_x_ad**2)
    dist_3_ad = abs(intercept_backbone_ad - intercept_CX)
    theta_ad = np.arccos((dist_backbone_ad**2 + dist_CX**2 - dist_3_ad**2) / (2 * dist_backbone_ad * dist_CX))
    angle_ad[int(frame.frame)] = np.rad2deg(theta_ad)
    mobile_ad = np.array([(backbone_indep['DC 7899'] + backbone_indep['DG 7892'] + backbone_indep['DC3 7901']).center_of_geometry()[2]])
    ref_ad = np.array([sel_CX.center_of_geometry()[2]])
    dist_ad[int(frame.frame)] = rmsd(mobile_ad, ref_ad)

In [ ]:
#for frame in u.trajectory:
#    print((backbone_indep['DC3 7901'] + backbone_indep['DC 7900'] + backbone_indep['DA 7895'] + backbone_indep[] + backbone_indep[] + backbone_indep[] + backbone_indep[]).center_of_geometry()[2])

In [ ]:
angle_ad

In [ ]:
dist_ad

In [ ]:
fig1 = plt.figure()
fig1.suptitle('Angle formed by adsorbed backbone with the graphene molecule frame by frame', fontsize=14, fontweight='bold')
plt.bar(frame_x, angle_ad.values())
plt.show()
    
fig2 = plt.figure()
fig2.suptitle('Distance between adsorbed backbone and the graphene molecule frame by frame', fontsize=14, fontweight = 'bold')
plt.plot(frame_x, dist_ad.values())
plt.show()
    


In [9]:
x_bkb = {}
y_bkb = {}
angle_bkb = {}
dist_bkb = {}
for frame in u.trajectory:
    x_bkb[int(frame.frame)] = []
    y_bkb[int(frame.frame)] = []
    for i in range(len(sel_DNA_backbone.positions)):
        x_bkb[int(frame.frame)].append(sel_DNA_backbone.positions[i][0])
        y_bkb[int(frame.frame)].append(sel_DNA_backbone.positions[i][2])
  #      x_ad.append(float(backbone_indep[key].positions[i][0]) for i in range(len(backbone_indep[key].positions)))
  #      y_ad.append(float(backbone_indep[key].positions[i][2]) for i in range(len(backbone_indep[key].positions)))
    x2D_bkb = np.asarray(x_bkb[int(frame.frame)])
    y2D_bkb = np.asarray(y_bkb[int(frame.frame)])
    slope_backbone_bkb, intercept_backbone_bkb, r_value_backbone_bkb, p_value_backbone_bkb, std_err_backbone_bkb = stats.linregress(x2D_bkb, y2D_bkb)
    common_x_bkb = float(intercept_backbone_bkb - intercept_CX) / (slope_CX - slope_backbone_bkb)
    dist_backbone_bkb = math.sqrt((slope_backbone_bkb * common_x_bkb)**2 + common_x_bkb**2)
    dist_CX = math.sqrt(abs((slope_CX * common_x_bkb) + common_x_bkb**2))
    dist_3_bkb = abs(intercept_backbone_bkb - intercept_CX)
    theta_bkb = np.arccos((dist_backbone_bkb**2 + dist_CX**2 - dist_3_bkb**2) / (2 * dist_backbone_bkb * dist_CX))
    angle_bkb[int(frame.frame)] = np.rad2deg(theta_bkb)
    mobile_bkb = np.array([sel_DNA_backbone.center_of_geometry()[2]])
    ref_bkb = np.array([sel_CX.center_of_geometry()[2]])
    dist_bkb[int(frame.frame)] = rmsd(mobile_bkb, ref_bkb)

/home/smuraru/Software/anaconda3/envs/MDA/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in arccos


In [ ]:
(slope_CX * common_x_bkb) + common_x_bkb**2

In [ ]:
frame_x = np.asarray([int(str(frame.frame)) for frame in u.trajectory])


In [ ]:
import csv


In [ ]:


with open('Distance2.csv', mode='w') as dist_file:
    writer = csv.writer(dist_file, delimiter=' ', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for frame in u.trajectory:
        writer.writerow([frame_x[int(frame.frame)], dist_bkb[int(frame.frame)]])
        if (int(frame.frame) % 1000 == 0):
            print("frame: ", int(frame.frame), "out of ", len(u.trajectory))



In [ ]:
with open('Angle2.csv', mode='w') as angle_file:
    writer = csv.writer(angle_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for frame in u.trajectory:
        writer.writerow([frame_x[int(frame.frame)], angle_bkb[int(frame.frame)]])


In [ ]:
angle_bkb

In [5]:
filename = "index_ions.ndx"
index_file = mda.selections.gromacs.SelectionWriter(filename, mode='w', numterms=None, preamble=None )
index_file.write(sel_MG, number=None, name='MG', frame=None, mode=None)
index_file.write(sel_CL, number=None, name='CL', frame=None, mode=None)
index_file.close()



In [ ]:
frame_x = np.asarray([int(str(frame.frame)) for frame in u.trajectory])

fig1 = plt.figure()
fig1.suptitle('Angle formed by ssDNA backbone with the graphene molecule frame by frame', fontsize=14, fontweight='bold')
plt.bar(frame_x, angle_bkb.values())
plt.show()
    
fig2 = plt.figure()
fig2.suptitle('Distance between ssDNA backbone and the graphene molecule frame by frame', fontsize=14, fontweight = 'bold')
plt.plot(frame_x, dist_bkb.values())
plt.show()